# Entanglement Cost

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/application/entangle/entanglement_cost.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This tutorial is to re-produce partial numerical results in the paper: "Entanglement Cost under Positive-Partial-Transpose-Preserving Operations" [doi-link](https://doi.org/10.1103/PhysRevLett.90.027901)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi

np_rng = np.random.default_rng()

## Bi-negativity

First, let's define the collection of all density matrix of dimension $d$ as $\mathcal{D}_d$.

$$ \mathcal{D}^{d}=\left\{ \rho\in\mathbb{C}^{d\times d}:\rho\succeq0,\mathrm{Tr}[\rho]=1\right\} $$

For a bipartite density matrix $\rho\in\mathcal{D}^{d_{A}\times d_{B}}$, its bi-negativity is defined as $\left|\rho^{\Gamma}\right|^{\Gamma}$. Let's explain the notation here:

1. partial transpose $\rho^\Gamma$. The explicit matrix operation can be written as

   $$ \left(\sum_{ij\mu\nu}\rho_{i\mu,j\nu}|i\mu\rangle\langle j\nu|\right)^{\Gamma}=\sum_{ij\mu\nu}\rho_{i\nu,j\mu}|i\mu\rangle\langle j\nu| $$

   where $|i\rangle$ and $|j\rangle$ are the basis of the Hilbert space of system $A$, and $|\mu\rangle$ and $|\nu\rangle$ are the basis of the Hilbert space of system $B$.

2. absolute value of a matrix: given a Hermitian matrix $A=A^\dagger$ with eigenvalue decomposition $A=U\Sigma U^\dagger$, then its absolute value is defined as $|A| = U|\Sigma|U^\dagger$. Here $|\Sigma|$ is the diagonal matrix with the absolute value of the eigenvalues of $A$. Be careful that this is NOT the same as element-wise absolute value.

Here are some properties of bi-negativity:

**Theorem** the bi-negativity of two-qubit states are all positive [link](https://oqp.iqoqi.oeaw.ac.at/qubit-bi-negativity) [arxiv-link](https://arxiv.org/abs/quant-ph/0308056)

$$ \rho\in\mathcal{D}^{2\times2}\;\Rightarrow\;\left|\rho^{\Gamma}\right|^{\Gamma}\succeq 0 $$

Let's verify this theorem by calculating the bi-negativity of some random two-qubit states.


In [ ]:
rho = numqi.random.rand_density_matrix(4)
print('density matrix:\n', np.round(rho,3))

rho_bineg = numqi.entangle.get_binegativity(rho, 2, 2)
print('binegativity of density matrix:\n', np.round(rho_bineg,3))

EVL = np.linalg.eigvalsh(rho_bineg)
print('eigenvalues of binegativity:', EVL)

assert EVL[0]>=0 # binegativity is positive for two qubits


**Theorem**: the bi-negativity of all Werner states are positive [doi-link](https://doi.org/10.1103/PhysRevLett.90.027901)

In [ ]:
dim = 3
alpha = np_rng.uniform(-1, 1)
rho = numqi.state.Werner(dim, alpha)

rho_bineg = numqi.entangle.get_binegativity(rho, dim, dim)
EVL = np.linalg.eigvalsh(rho_bineg)
print('eigenvalues of binegativity:', EVL[:4])
assert EVL[0]>=0 # binegativity is positive for Werner state

## Optimization of bi-negativity

A natural question is what is the most "negative" density matrix in bi-negativity? Or say solving the following optimization problem:

$$ \min_{\rho\in\mathcal{D}^{d_{A}\times d_{B}}}\lambda_{1}\left[\left|\rho^{\Gamma}\right|^{\Gamma}\right] $$

with $\lambda_{1}$ being the smallest eigenvalue. This is a typical manifold optimization problem, and can be addressed in `numqi` with the following code:

In [ ]:
dimA = 3
dimB = 3
model = numqi.entangle.SearchMinimumBinegativityModel(dimA, dimB)
theta_optim = numqi.optimize.minimize(model, num_repeat=20, tol=1e-7, print_every_round=1)
rho = model.manifold_rho().detach().numpy().copy()
print('optimal value (minimum eigenvalue):', theta_optim.fun)

Below list the optimal value for different dimensions:

| $d_A\times d_B$ | optimal value |
| :-: | :-: |
| $2\times 2$ | `3.8e-7` |
| $2\times 3$ | `-0.05392654084929269` |
| $2\times 4$ | `-0.053925646486662315` |
| $3\times 3$ | `-0.17677641180960016` |

## PPT entanglement cost

In the paper: "Entanglement Cost under Positive-Partial-Transpose-Preserving Operations" [doi-link](https://doi.org/10.1103/PhysRevLett.90.027901), PPT entanglement cost $E_{\mathrm{PPT}}(\rho)$ is given both lower and upper bounds.

$$ \log_{2}\mathrm{tr}\left|\rho^{\Gamma}\right|\leq E_{\mathrm{PPT}}(\rho)\leq\log_{2}Z\left(\rho\right) $$

$$ Z\left(\rho\right)=\mathrm{tr}\left|\rho^{\Gamma}\right|+d_{A}d_{B}\max\left(0,-\lambda_{1}\left[\left|\rho^{\Gamma}\right|^{\Gamma}\right]\right) $$

Since the upper bound matches the lower bound for all Werner states, the PPT entanglement cost is exactly known for Werner states.

Below draw the PPT entanglement cost of Werner states of different dimensions.

In [ ]:
dim_list = [2,3,4,5]
dim = 3
alpha_list = np.linspace(0, 1, 100)
z0 = []
for dim in dim_list:
    z0.append([numqi.entangle.get_PPT_entanglement_cost_bound(numqi.state.Werner(dim, x), dim, dim)[0] for x in alpha_list])
z0 = np.stack(z0, axis=0)

fig,ax = plt.subplots()
for ind0 in range(len(dim_list)):
    ax.plot(alpha_list, z0[ind0], label=f'dim={dim_list[ind0]}')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('PPT entanglement cost')
ax.legend()
fig.tight_layout()
